In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import wplotlib
from wuml import HSIC

In [4]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', randomly_shuffle_batch=True,
					batch_size=20, label_type='discrete', encode_discrete_label_to_one_hot=True )

Remember that you are trying to "minimize" this objective

In [5]:
def costFunction(x, y, ŷ, ind):
	H = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	# compare the HSIC value against numpy library
	#y = y.detach().cpu().numpy()
	#ŷ = ŷ.detach().cpu().numpy()
	#H2 = HSIC(ŷ, y, X_kernel='linear', Y_kernel='linear', sigma_type='mpd' )
	#import pdb; pdb.set_trace()
	return -H  

----------------------------------------------<br>
Create basic network and train

In [6]:
bNet = wuml.basicNetwork(costFunction, data, networkStructure=[(30,'relu'),('bn', True), (50,'relu'),('bn', True),(3,'none')], max_epoch=500, learning_rate=0.001)
bNet.train(print_status=True)
Ŷ = bNet(data, output_type='ndarray')		#Takes Numpy array or Tensor as input and outputs a Tensor

In [7]:
Network_out_v_Label = np.hstack((Ŷ, data.Y))
wuml.jupyter_print(Network_out_v_Label)

,0,1,2,3,4,5
0,0.836606,0.558910,1.115963,1.0,0.0,0.0
1,0.855651,0.214034,1.049345,1.0,0.0,0.0
2,0.731848,0.522496,1.413070,1.0,0.0,0.0
3,0.436627,1.364961,2.268159,1.0,0.0,0.0
4,0.438976,-0.155353,-0.155826,1.0,0.0,0.0
...,...,...,...,...,...,...
173,0.822058,-0.425618,0.139526,0.0,0.0,1.0
174,0.817423,-0.405581,0.037105,0.0,0.0,1.0
175,0.653430,0.166129,0.149205,0.0,0.0,1.0
176,0.625715,0.144855,0.131415,0.0,0.0,1.0
